In [68]:
"""
    Código python para hacer un Exploratory Data Analysis que nos permita entender la información obtenida
    con el objetivo de proponer una estrategia para clasificar los sellers que representen importancia para
    la línea de negocio. Para usar este código es importante primero haber obtenido los datos de la API
    mediante el file challenge_getData.ipynb
"""
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df_items = pd.read_csv('dataset.csv')
df_categories = pd.read_csv('categories.csv')




In [13]:
df_item_categories = pd.merge(df_categories.rename(columns={'id': 'categoria_id','name': 'category_name'}),df_items,how='inner',left_on='categoria_id', right_on='original_category')

In [61]:
df_item_categories

,categoria_id,category_name,id,title,condition,permalink,accepts_mercadopago,listing_type_id,site_id,original_category,...,installments_quantity,installments_amount,installments_rate,seller_id,seller_nickname,official_store_id,official_store_name,available_quantity,use_thumbnail_id,installments_rate_normalized
0,MLA5725,Accesorios para Vehículos,MLA1583153780,Microfibra 40x80 Cm Autos Detailing Secado Rap...,new,https://www.mercadolibre.com.ar/microfibra-40x...,True,gold_special,MLA,MLA5725,...,NaN,NaN,NaN,290403581,LS PALERMO,NaN,NaN,500,True,0.000000
1,MLA5725,Accesorios para Vehículos,MLA1277641650,Pack 4 Paños Microfibra Multiuso Limpieza Auto...,new,https://www.mercadolibre.com.ar/pack-4-panos-m...,True,gold_special,MLA,MLA5725,...,NaN,NaN,NaN,702150429,GLOBALNAVET,NaN,NaN,150,True,0.000000
2,MLA5725,Accesorios para Vehículos,MLA1378229931,Casco Moto Vertigo V50 Dark Edicion Especial. ...,new,https://articulo.mercadolibre.com.ar/MLA-13782...,True,gold_special,MLA,MLA5725,...,6.0,13935.98,12.74,263770607,GRAVEDADX,NaN,NaN,50,False,0.634538
3,MLA5725,Accesorios para Vehículos,MLA851109741,Carcasa Llave Navaja Peugeot 2 Bot 307 308 C/...,new,https://articulo.mercadolibre.com.ar/MLA-85110...,True,gold_special,MLA,MLA5725,...,6.0,1670.05,34.86,435859328,KEYFAD,3272.0,Keyfad,50,True,0.000000
4,MLA5725,Accesorios para Vehículos,MLA1148567142,Bateria Alpina 12n5-3b Gel Zanella Zb 110 125 ...,new,https://articulo.mercadolibre.com.ar/MLA-11485...,True,gold_special,MLA,MLA5725,...,6.0,4506.41,12.74,685241555,ALPINA ARG,2541.0,Alpina,250,True,0.634538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33033,MLA1953,Otras categorías,MLA1439582821,Sahumerios Bangladesh Simple Empaste X100 Unid...,new,https://www.mercadolibre.com.ar/sahumerios-ban...,True,gold_special,MLA,MLA1953,...,NaN,NaN,NaN,191914203,G.UTOPIA,54275.0,Grupo Utopia,150,True,0.000000
33034,MLA1953,Otras categorías,MLA1389640875,Raw Armador De Conos Automatico Six Shooter 6 ...,new,https://www.mercadolibre.com.ar/raw-armador-de...,True,gold_special,MLA,MLA1953,...,6.0,10451.65,34.86,251681571,PLANTOLUEGOEXISTOGROWSHOP,54445.0,Planto Luego Existo,1,True,0.000000
33035,MLA1953,Otras categorías,MLA1457772836,Sahumerios Sagrada Madre Artesano Natural Inci...,new,https://www.mercadolibre.com.ar/sahumerios-sag...,True,gold_special,MLA,MLA1953,...,6.0,1231.72,34.86,191914203,G.UTOPIA,54275.0,Grupo Utopia,1,True,0.000000
33036,MLA1953,Otras categorías,MLA1705231190,Pastilla Sagrada Madre 7 Dias De Limpieza Frag...,new,https://www.mercadolibre.com.ar/pastilla-sagra...,True,gold_special,MLA,MLA1953,...,NaN,NaN,NaN,789683079,ORIENTE_MISTICO,NaN,NaN,1,True,0.000000


In [83]:
#hacemos un analisis de los quantity installments que hay
installment_quantity_df = df_item_categories.groupby(['installments_quantity'])['id'].count()

installment_quantity_df = installment_quantity_df.reset_index()
installment_quantity_df.columns = ['installments_quantity', 'product_count']

#En caso de querer analizarlo en excel podemos exportarlo
#installment_quantity_df.to_csv('porductos_por_categoria_sellerID_listingType.csv', encoding='utf-8', index=False)

# Mostramos el resultado
print(installment_quantity_df)

   installments_quantity  product_count
0                    1.0             41
1                    3.0            925
2                    6.0          25540
3                    9.0            426
4                   12.0            600
5                   18.0             39


In [84]:
#Vamos a calcular un score para installments quantity donde el seller que de 18 meses tendrá una calificación de 1 y los demás
#se basarán en el cálculo del score
min_installments = installment_quantity_df['installments_quantity'].min()
max_installments = installment_quantity_df['installments_quantity'].max()

#Calculamos el score
installment_quantity_df['score'] = (installment_quantity_df['installments_quantity'] - min_installments) / (max_installments - min_installments)

installment_quantity_df['score'] = np.floor(installment_quantity_df['score'] * 10) / 10

# Mostrar el resultado
installment_quantity_df

,installments_quantity,product_count,score
0,1.0,41,0.0
1,3.0,925,0.1
2,6.0,25540,0.2
3,9.0,426,0.4
4,12.0,600,0.6
5,18.0,39,1.0


In [97]:
#hacemos un analisis de los quantity rates que hay
installment_rate_df = df_item_categories.groupby(['installments_rate'])['id'].count()

installment_rate_df = installment_rate_df.reset_index()
installment_rate_df.columns = ['installments_rate', 'product_count']

#En caso de querer analizarlo en excel podemos exportarlo
#installment_rate_df.to_csv('porductos_por_categoria_sellerID_listingType.csv', encoding='utf-8', index=False)

# Mostramos el resultado
print(installment_rate_df)

   installments_rate  product_count
0               0.00           3652
1              11.41           1305
2              12.74            866
3              34.86          21748


In [ ]:
#Vamos a calcular un score para installments quantity donde el seller que de 18 meses tendrá una calificación de 1 y los demás
#se basarán en el cálculo del score
min_installments_rate = installment_rate_df['installments_rate'].min()
max_installments_rate = installment_rate_df['installments_rate'].max()

#Calculamos el score
installment_rate_df['score'] = (max_installments_rate - installment_rate_df['installments_rate']) / (max_installments_rate - min_installments_rate)

installment_rate_df['score'] = np.floor(installment_rate_df['score'] * 10) / 10

# Mostrar el resultado
print(installment_rate_df)

   installments_rate  product_count  score
0               0.00           3652    1.0
1              11.41           1305    0.6
2              12.74            866    0.6
3              34.86          21748    0.0


"\n#Vamos a calcular un score para installments rate donde el seller que de la menor tasa tendrá el menor score y de ahí se va a calcular los demás\nmin_rate = df_item_categories['installments_rate'].min()  # Vendedor con el menor Installments Rate\nmax_rate = df_item_categories['installments_rate'].max()  # Vendedor con el mayor Installments Rate\n\n# \ndf_item_categories['installments_rate_normalized'] = (max_rate - df_item_categories['installments_rate']) / (max_rate - min_rate)\n\n# Paso 3: Rellenar valores NaN con un valor de 0 (o 1 dependiendo de la lógica)\ndf_item_categories['installments_rate_normalized'].fillna(0)\n\n# Verifica el resultado\ndf_item_categories[['seller_id', 'installments_rate', 'installments_rate_normalized']]\ndf_item_categories.to_csv('rate_installmentsRates.csv', index=False)\n"

In [94]:
# Agrupar por seller_id y contar categorías únicas
sellers_most_categories = df_item_categories.groupby('seller_id')['category_name'].nunique().reset_index(name='category_count')

# Ordenar de mayor a menor cantidad de categorías
sellers_most_categories = sellers_most_categories.sort_values(by='category_count', ascending=False)
print(sellers_most_categories)

       seller_id  category_count
847     37133205              24
345     13647702              20
890     39144101              20
4067   188316739              18
56       1176849              15
...          ...             ...
8534  2109015579               1
8535  2111316385               1
8536  2111827077               1
8537  2115481041               1
7         110162               1

[8555 rows x 2 columns]


In [95]:
#Vamos a calcular un score para la cantidad de productos que un seller tiene en categorías distintas, si un seller tiene diversidad
#tiene más probabilidad de que le compren productos
min_sellers_most_categories = sellers_most_categories['category_count'].min()
max_sellers_most_categories = sellers_most_categories['category_count'].max()

#Calculamos el score
sellers_most_categories['score'] = (sellers_most_categories['category_count'] - min_sellers_most_categories) / (max_sellers_most_categories - min_sellers_most_categories)

sellers_most_categories['score'] = np.floor(sellers_most_categories['score'] * 10) / 10

# Mostrar el resultado
print(sellers_most_categories)

       seller_id  category_count  score
847     37133205              24    1.0
345     13647702              20    0.8
890     39144101              20    0.8
4067   188316739              18    0.7
56       1176849              15    0.6
...          ...             ...    ...
8534  2109015579               1    0.0
8535  2111316385               1    0.0
8536  2111827077               1    0.0
8537  2115481041               1    0.0
7         110162               1    0.0

[8555 rows x 3 columns]


In [112]:
#hacemos un analisis de los listing type que hay. vamos a dejar fuera a inmuebles y servicios
filter_listing_type_df = df_item_categories[~df_item_categories['category_name'].isin(['Inmuebles','Servicios'])]
filter_listing_type_df = df_item_categories[df_item_categories['listing_type_id'].isin(['gold_pro'])]
listing_type_df = filter_listing_type_df.groupby(['seller_id'])['id'].count()

listing_type_df = listing_type_df.reset_index()
listing_type_df.columns = ['seller_id', 'product_count']

#Como los valores son demasiados para asignar un score por seller, vamos a agruparlos en bins con etiquetas que indiquen
#si el seller tiene casi ningún producto en gold_pro, pocos, aceptables o muchos
# un seller con muchos productos en gold_pro puede ser más atractivo 
listing_type_df['value_bins'] = pd.cut(listing_type_df['product_count'], bins=[0, 20, 40, 100, 200],  labels=['Casi Nada', 'Pocos', 'Aceptable', 'Muchos'])

# los valores en casi nada van a ser ignorados para que el cálculo del score no esté sesgado, de cualquier manera se
# les asignará un score de 0
filtered_listing_type_df = listing_type_df[~listing_type_df['value_bins'].isin(['Casi Nada'])]

group_scores = (
    filtered_listing_type_df.groupby('value_bins')['product_count']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

group_scores['score'] = group_scores['product_count'] / group_scores['product_count'].max()

score_mapping = group_scores.set_index('value_bins')['score'].to_dict()
filtered_listing_type_df['group_score'] = filtered_listing_type_df['value_bins'].map(score_mapping).fillna(0)

# Mostramos el resultado
print(listing_type_df)

       seller_id  product_count value_bins
0          79704              1  Casi Nada
1         111286              1  Casi Nada
2         111621              3  Casi Nada
3         112133              1  Casi Nada
4         135942              2  Casi Nada
...          ...            ...        ...
1210  2073405247              1  Casi Nada
1211  2092994529              1  Casi Nada
1212  2121491235              1  Casi Nada
1213  2163881647              1  Casi Nada
1214  2170682452              4  Casi Nada

[1215 rows x 3 columns]


C:\Users\em_ba\AppData\Local\Temp\ipykernel_10700\1712012904.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  filtered_listing_type_df.groupby('value_bins')['product_count']
C:\Users\em_ba\AppData\Local\Temp\ipykernel_10700\1712012904.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_listing_type_df['group_score'] = filtered_listing_type_df['value_bins'].map(score_mapping).fillna(0)
